In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import ast

In [2]:
# Import word embeddings and reformat
import gensim
from gensim.models.keyedvectors import KeyedVectors
word_vectors = gensim.models.Word2Vec.load('nearby_embeddings2')
wordVectors = []
for word in word_vectors.wv.index2word:
    wordVectors.append(word_vectors.wv[word])
wordVectors = np.array(wordVectors)
wordsList = word_vectors.wv.index2word

max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [6]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [7]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [8]:
len(raw_data), len(y_list)

(16623, 16623)

In [9]:
word_tokenize(raw_data['body'].iloc[105]), raw_data['original_text'].iloc[105], y_list[105]

(['Until', 'that', 'u', 'can', 'send', 'next', 'property'],
 ['property'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
numFiles = len(raw_data)
ids = np.zeros((numFiles, max_sequence_length), dtype='int32')
fileCounter = 0
indexCounter = 0

for line in raw_data['body']:
    split = word_tokenize(line)
    indexCounter = 0
    
    for word in split:
       # print(indexCounter)
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = len(wordsList)-2 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= max_sequence_length:
            break
    fileCounter = fileCounter + 1 
    print(fileCounter/numFiles)

6.0157612945918305e-05
0.00012031522589183661
0.00018047283883775492
0.00024063045178367322
0.00030078806472959155
0.00036094567767550983
0.00042110329062142816
0.00048126090356734644
0.0005414185165132648
0.0006015761294591831
0.0006617337424051013
0.0007218913553510197
0.000782048968296938
0.0008422065812428563
0.0009023641941887746
0.0009625218071346929
0.0010226794200806112
0.0010828370330265296
0.0011429946459724479
0.0012031522589183662
0.0012633098718642843
0.0013234674848102027
0.001383625097756121
0.0014437827107020393
0.0015039403236479577
0.001564097936593876
0.0016242555495397943
0.0016844131624857127
0.0017445707754316308
0.001804728388377549
0.0018648860013234674
0.0019250436142693858
0.001985201227215304
0.0020453588401612224
0.0021055164531071408
0.002165674066053059
0.0022258316789989774
0.0022859892919448958
0.002346146904890814
0.0024063045178367324
0.0024664621307826503
0.0025266197437285687
0.002586777356674487
0.0026469349696204053
0.0027070925825663237
0.00276725

0.02586777356674487
0.02592793117969079
0.02598808879263671
0.026048246405582627
0.026108404018528545
0.026168561631474463
0.02622871924442038
0.0262888768573663
0.026349034470312217
0.026409192083258138
0.026469349696204056
0.026529507309149974
0.02658966492209589
0.02664982253504181
0.026709980147987727
0.026770137760933645
0.026830295373879563
0.02689045298682548
0.026950610599771403
0.02701076821271732
0.02707092582566324
0.027131083438609156
0.027191241051555074
0.027251398664500992
0.02731155627744691
0.027371713890392828
0.02743187150333875
0.027492029116284667
0.027552186729230585
0.027612344342176503
0.02767250195512242
0.02773265956806834
0.027792817181014257
0.027852974793960174
0.027913132406906092
0.027973290019852014
0.02803344763279793
0.02809360524574385
0.028153762858689767
0.028213920471635685
0.028274078084581603
0.02833423569752752
0.02839439331047344
0.02845455092341936
0.02851470853636528
0.028574866149311196
0.028635023762257114
0.028695181375203032
0.02875533898

0.09715454490765807
0.09721470252060398
0.0972748601335499
0.09733501774649582
0.09739517535944174
0.09745533297238766
0.09751549058533357
0.09757564819827949
0.09763580581122541
0.09769596342417133
0.09775612103711724
0.09781627865006316
0.09787643626300908
0.097936593875955
0.09799675148890093
0.09805690910184683
0.09811706671479276
0.09817722432773868
0.0982373819406846
0.0982975395536305
0.09835769716657643
0.09841785477952235
0.09847801239246827
0.09853817000541419
0.0985983276183601
0.09865848523130602
0.09871864284425194
0.09877880045719786
0.09883895807014377
0.09889911568308969
0.09895927329603561
0.09901943090898153
0.09907958852192746
0.09913974613487336
0.09919990374781928
0.0992600613607652
0.09932021897371113
0.09938037658665703
0.09944053419960296
0.09950069181254888
0.0995608494254948
0.09962100703844072
0.09968116465138663
0.09974132226433255
0.09980147987727847
0.09986163749022439
0.0999217951031703
0.09998195271611622
0.10004211032906214
0.10010226794200806
0.1001624

0.15237923359201108
0.15243939120495698
0.1524995488179029
0.15255970643084882
0.15261986404379474
0.15268002165674066
0.15274017926968658
0.1528003368826325
0.15286049449557843
0.15292065210852435
0.15298080972147024
0.15304096733441616
0.15310112494736208
0.153161282560308
0.15322144017325393
0.15328159778619985
0.15334175539914577
0.1534019130120917
0.1534620706250376
0.1535222282379835
0.15358238585092943
0.15364254346387535
0.15370270107682127
0.1537628586897672
0.1538230163027131
0.15388317391565903
0.15394333152860495
0.15400348914155088
0.15406364675449677
0.1541238043674427
0.1541839619803886
0.15424411959333453
0.15430427720628045
0.15436443481922638
0.1544245924321723
0.15448475004511822
0.15454490765806414
0.15460506527101003
0.15466522288395596
0.15472538049690188
0.1547855381098478
0.15484569572279372
0.15490585333573964
0.15496601094868556
0.15502616856163148
0.1550863261745774
0.1551464837875233
0.15520664140046922
0.15526679901341514
0.15532695662636106
0.1553871142393

0.2129579498285508
0.2130181074414967
0.21307826505444263
0.21313842266738856
0.21319858028033448
0.2132587378932804
0.21331889550622632
0.21337905311917224
0.21343921073211816
0.21349936834506406
0.21355952595800998
0.2136196835709559
0.21367984118390182
0.21373999879684774
0.21380015640979366
0.21386031402273958
0.2139204716356855
0.21398062924863143
0.21404078686157732
0.21410094447452324
0.21416110208746916
0.21422125970041508
0.214281417313361
0.21434157492630693
0.21440173253925285
0.21446189015219877
0.2145220477651447
0.21458220537809058
0.2146423629910365
0.21470252060398243
0.21476267821692835
0.21482283582987427
0.2148829934428202
0.2149431510557661
0.21500330866871203
0.21506346628165796
0.21512362389460385
0.21518378150754977
0.2152439391204957
0.2153040967334416
0.21536425434638753
0.21542441195933346
0.21548456957227938
0.2155447271852253
0.21560488479817122
0.2156650424111171
0.21572520002406304
0.21578535763700896
0.21584551524995488
0.2159056728629008
0.21596583047584

...

0.7338025627143115
0.7338627203272574
0.7339228779402033
0.7339830355531493
0.7340431931660951
0.7341033507790411
0.734163508391987
0.7342236660049329
0.7342838236178788
0.7343439812308248
0.7344041388437706
0.7344642964567166
0.7345244540696625
0.7345846116826085
0.7346447692955543
0.7347049269085003
0.7347650845214462
0.7348252421343922
0.734885399747338
0.734945557360284
0.7350057149732299
0.7350658725861757
0.7351260301991217
0.7351861878120676
0.7352463454250135
0.7353065030379594
0.7353666606509054
0.7354268182638513
0.7354869758767972
0.7355471334897431
0.7356072911026891
0.7356674487156349
0.7357276063285809
0.7357877639415268
0.7358479215544728
0.7359080791674186
0.7359682367803646
0.7360283943933105
0.7360885520062564
0.7361487096192023
0.7362088672321482
0.7362690248450942
0.73632918245804
0.736389340070986
0.7364494976839319
0.7365096552968778
0.7365698129098237
0.7366299705227697
0.7366901281357155
0.7367502857486615
0.7368104433616074
0.7368706009745534
0.7369307585874992

0.7812067617156951
0.781266919328641
0.781327076941587
0.7813872345545329
0.7814473921674788
0.7815075497804247
0.7815677073933707
0.7816278650063165
0.7816880226192625
0.7817481802322084
0.7818083378451544
0.7818684954581002
0.7819286530710461
0.7819888106839921
0.7820489682969379
0.7821091259098839
0.7821692835228298
0.7822294411357757
0.7822895987487216
0.7823497563616676
0.7824099139746135
0.7824700715875594
0.7825302292005053
0.7825903868134513
0.7826505444263971
0.7827107020393431
0.782770859652289
0.782831017265235
0.7828911748781808
0.7829513324911268
0.7830114901040727
0.7830716477170185
0.7831318053299645
0.7831919629429104
0.7832521205558564
0.7833122781688022
0.7833724357817482
0.7834325933946941
0.78349275100764
0.7835529086205859
0.7836130662335319
0.7836732238464778
0.7837333814594237
0.7837935390723696
0.7838536966853156
0.7839138542982614
0.7839740119112074
0.7840341695241533
0.7840943271370991
0.7841544847500451
0.784214642362991
0.784274799975937
0.7843349575888828
0

0.8410034289839379
0.8410635865968838
0.8411237442098297
0.8411839018227757
0.8412440594357216
0.8413042170486675
0.8413643746616134
0.8414245322745594
0.8414846898875052
0.8415448475004512
0.8416050051133971
0.8416651627263431
0.8417253203392889
0.8417854779522349
0.8418456355651808
0.8419057931781266
0.8419659507910726
0.8420261084040185
0.8420862660169645
0.8421464236299103
0.8422065812428563
0.8422667388558022
0.8423268964687481
0.842387054081694
0.84244721169464
0.8425073693075859
0.8425675269205318
0.8426276845334777
0.8426878421464237
0.8427479997593695
0.8428081573723155
0.8428683149852614
0.8429284725982074
0.8429886302111532
0.8430487878240991
0.8431089454370451
0.8431691030499909
0.8432292606629369
0.8432894182758828
0.8433495758888288
0.8434097335017746
0.8434698911147206
0.8435300487276665
0.8435902063406124
0.8436503639535583
0.8437105215665043
0.8437706791794501
0.8438308367923961
0.843890994405342
0.843951152018288
0.8440113096312338
0.8440714672441797
0.844131624857125

0.8870240028875654
0.8870841605005113
0.8871443181134573
0.8872044757264032
0.8872646333393491
0.887324790952295
0.887384948565241
0.8874451061781868
0.8875052637911328
0.8875654214040787
0.8876255790170247
0.8876857366299705
0.8877458942429164
0.8878060518558624
0.8878662094688082
0.8879263670817542
0.8879865246947001
0.888046682307646
0.8881068399205919
0.8881669975335379
0.8882271551464838
0.8882873127594297
0.8883474703723756
0.8884076279853216
0.8884677855982674
0.8885279432112134
0.8885881008241593
0.8886482584371053
0.8887084160500511
0.888768573662997
0.888828731275943
0.8888888888888888
0.8889490465018348
0.8890092041147807
0.8890693617277267
0.8891295193406725
0.8891896769536185
0.8892498345665644
0.8893099921795103
0.8893701497924562
0.8894303074054022
0.889490465018348
0.889550622631294
0.8896107802442399
0.8896709378571859
0.8897310954701317
0.8897912530830777
0.8898514106960236
0.8899115683089694
0.8899717259219154
0.8900318835348613
0.8900920411478073
0.8901521987607531


0.927871022077844
0.9279311796907899
0.9279913373037358
0.9280514949166817
0.9281116525296276
0.9281718101425736
0.9282319677555194
0.9282921253684654
0.9283522829814113
0.9284124405943572
0.9284725982073031
0.9285327558202491
0.9285929134331949
0.9286530710461409
0.9287132286590868
0.9287733862720328
0.9288335438849786
0.9288937014979246
0.9289538591108705
0.9290140167238164
0.9290741743367623
0.9291343319497082
0.9291944895626542
0.9292546471756
0.929314804788546
0.9293749624014919
0.9294351200144378
0.9294952776273837
0.9295554352403297
0.9296155928532756
0.9296757504662215
0.9297359080791674
0.9297960656921134
0.9298562233050592
0.9299163809180052
0.9299765385309511
0.9300366961438971
0.9300968537568429
0.9301570113697889
0.9302171689827348
0.9302773265956806
0.9303374842086266
0.9303976418215725
0.9304577994345185
0.9305179570474643
0.9305781146604103
0.9306382722733562
0.9306984298863021
0.930758587499248
0.930818745112194
0.9308789027251398
0.9309390603380858
0.9309992179510317


In [11]:
np.save('nearby_ids2',ids)

In [12]:
ids = np.load('nearby_ids2.npy')

In [13]:
batchSize = 100
lstmUnits = 64
numClasses = 2
epochs = 20
numDimensions = 100
train_len = 14000

X_train = ids[:train_len]
y_train = y_list[:train_len]
X_test = ids[train_len:]
y_test = y_list[train_len:]

In [14]:
len(X_train), len(y_test)

(14000, 2623)

In [15]:
for each in y_test:
    print(each[0])

[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]
[1. 0.]
[1. 0.]
[0. 1.]
[1. 0.]


In [16]:
def getTrainBatch(j):
    i = j*batchSize
    arr = X_train[i:i+batchSize]
    labels = y_train[i:i+batchSize]
    #print(labels[0])
    return arr, labels

In [17]:
def getTestBatch(j):
    i = j*batchSize
    arr = X_test[i:i+batchSize]
    labels =   y_test[i:i+batchSize]
    return arr, labels

In [18]:
getTrainBatch(1)[1].shape

(100, 20, 2)

In [19]:
import tensorflow as tf
tf.reset_default_graph()
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_sequence_length])

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
wordVectors = tf.convert_to_tensor(wordVectors, dtype=tf.float32)

In [21]:
data = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)


In [22]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [23]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data,dtype=tf.float32)

In [24]:
#print(outputs)
print(value2)
outputs = tf.concat(outputs, 2)
#print(outputs.shape)

((LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(100, 64) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_4:0' shape=(100, 64) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_5:0' shape=(100, 64) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/fw/fw/while/Exit_6:0' shape=(100, 64) dtype=float32>)), (LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(100, 64) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_4:0' shape=(100, 64) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_5:0' shape=(100, 64) dtype=float32>, h=<tf.Tensor 'bidirectional_rnn/bw/bw/while/Exit_6:0' shape=(100, 64) dtype=float32>)))


In [25]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [26]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)
#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
#init_g = tf.global_variables_initializer()
#init_l = tf.local_variables_initializer()
#with tf.Session() as sess:
    #tf.initialize_all_variables().run()
#    sess.run(init_g)
#    sess.run(init_l)
#    nextBatch, nextBatchLabels = getTrainBatch(2)
        #print(j/2)
#    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {input_data: nextBatch, labels: nextBatchLabels})
#    print(pa.shape, la.shape, fp, fn, tp, tn)
#    a = pred[0]
#    b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))
correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


In [27]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

(100, 20, 128) (128, 2) (100, 20, 2) (100, 20, 2)


In [29]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [30]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)
train_analysis = {'epoch': [], 'loss': [], 'true_positives': [], 'false_positives': [], 'false_negative': [], 'true_negatives':[],
                 'precision': [], 'recall': []}


for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(X_train)//batchSize):
        nextBatch, nextBatchLabels = getTrainBatch(j);
        #print(j/2)
        if i == epochs - 1:
            pass
            #ll, _,train_tp, train_fp, train_tn, train_fn, train_prec, train_rec = sess.run([loss, optimizer, true_positives, false_positives, true_negatives, false_negatives, prec, rec], {input_data: nextBatch, labels: nextBatchLabels})
        else:
            ll, _ = sess.run([loss, optimizer], {input_data: nextBatch, labels: nextBatchLabels})
        losses += ll


    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
#print(losses, train_tp, train_fp, train_tn, train_fn, train_prec, train_rec)
writer.close()

epoch:  0
saved to models/pretrained_lstm.ckpt-0
saved to models/pretrained_lstm.ckpt-0
20.893155977129936
epoch:  1
saved to models/pretrained_lstm.ckpt-1
saved to models/pretrained_lstm.ckpt-1
11.543441396206617
epoch:  2
saved to models/pretrained_lstm.ckpt-2
saved to models/pretrained_lstm.ckpt-2
10.347514376044273
epoch:  3
saved to models/pretrained_lstm.ckpt-3
saved to models/pretrained_lstm.ckpt-3
9.687595274299383
epoch:  4
saved to models/pretrained_lstm.ckpt-4
saved to models/pretrained_lstm.ckpt-4
9.136289551854134
epoch:  5
saved to models/pretrained_lstm.ckpt-5
saved to models/pretrained_lstm.ckpt-5
8.72517205402255
epoch:  6
saved to models/pretrained_lstm.ckpt-6
saved to models/pretrained_lstm.ckpt-6
8.478497814387083
epoch:  7
saved to models/pretrained_lstm.ckpt-7
saved to models/pretrained_lstm.ckpt-7
8.289330523461103
epoch:  8
saved to models/pretrained_lstm.ckpt-8
saved to models/pretrained_lstm.ckpt-8
8.132503762841225
epoch:  9
saved to models/pretrained_lstm.ck

In [31]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 2])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatch, nextBatchLabels = getTestBatch(j);
    pred, acc, tp, tn, fp, fn, p, r = sess.run([prediction, accuracy, true_positives, true_negatives, false_positives, false_negatives, prec, rec], {input_data: nextBatch, labels: nextBatchLabels})
    print(pred.shape)
    fin_pred[jj:jj+100] = pred

print(tp,tn,fp,fn,p,r)
    #    for i in range(10):

#        print(pred[i][:6])
#        print(nextBatchLabels[i][:6])
#        print('----------')

(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(4003.0, 4003.0) (44657.0, 44657.0) (880.0, 880.0) (460.0, 460.0) (0.8199659, 0.8197829) (0.89771134, 0.89693034)


# 

In [32]:
#fin_pred = np.argmax(fin_pred
fin_test = np.argmax(y_test[:test_batches*100], axis=2).shape

In [33]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(np.argmax(fin_pred, axis=2)[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [34]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [35]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [40]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(3999, 879, 437, 0.8198031980319803, 0.901487826871055, 0.8587073223105003)

In [41]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [42]:
prediction_analysis_pandas

,body,original_text,pred_original_text,false_positives,true_positives,false_negatives
0,"[i, call, on, just, dial, .., its, better, the...",[samsung],"[dial, better, samsung]","{better, dial}",{samsung},{}
1,"[i, want, ambala, bus, stand, context, no, .]","[bus, stand]","[ambala, bus, stand]",{ambala},"{stand, bus}",{}
2,"[Beauty, parlour, for, women]","[Beauty, parlour]",[parlour],{},{parlour},{Beauty}
3,"[Its, not, a, night, club, dear]","[night, club]","[night, club, dear]",{dear},"{club, night}",{}
4,"[More, Nearby, :, workshop]",[workshop],[workshop],{},{workshop},{}
5,"[Not, send, me, agai]",[agai],[agai],{},{agai},{}
6,"[Car, workshop]","[Car, workshop]","[Car, workshop]",{},"{workshop, Car}",{}
7,[Daraganj],[Daraganj],[Daraganj],{},{Daraganj},{}
8,"[Look, for, him]","[for, him]","[Look, him]",{Look},{him},{for}
9,"[Nearby, sbi, branch]","[sbi, branch]","[sbi, branch]",{},"{branch, sbi}",{}
